In [3]:
H = [2500, 3000, 4000, 5500, 4500, 3500]

6-element Vector{Int64}:
 2500
 3000
 4000
 5500
 4500
 3500

In [16]:
using JuMP, Cbc

m = Model()

@variable(m, x[1:6] >= 0) # of hired employees in period t
@variable(m, y[1:6] >= 0) # of retiring employees in period t
@variable(m, s[0:6] >= 0) # of carryover labor hours in from period t-1 to t
@variable(m, a[1:6] >= 0) # of hrs an employee works in period t
@variable(m, z[0:6] >= 0) # of current employees in period t

@objective(m, Min, 5000 * sum(x) + 10000 * sum(y) + 2000 * sum(z))

@constraint(m, curr_work[t in 1:6], z[t] == z[t-1] + x[t] - y[t])
@constraint(m, carryover[t in 1:6], s[t] == s[t-1] + (H[t] - (a[t] * z[t])))
@constraint(m, start_worker, z[0] == 35)
@constraint(m, start_carryover, s[0] == 0)
@constraint(m, max_retire, sum(y) <= 8)
@constraint(m, max_indv_labor[t in 1:6], a[t] <= 160)
@constraint(m, tot_hrs_meet, sum(a[t] * z[t] + s[t] for t in 1:6)== sum(H))
@constraint(m, max_carryover[t in 1:6], s[t] <= 400)
#@constraint(m, max_worker_hrs[t in 1:6], a[t] *  <= H[t])
@constraint(m, no_last_carryover, s[6] == 0)

no_last_carryover : s[6] == 0.0

In [17]:
set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

LoadError: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.EqualTo{Float64}` constraint is not supported by the model.